# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [46]:
from __future__ import division
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# Import everything needed to edit/save/watch video clips
# from moviepy.editor import VideoFileClip
# from IPython.display import HTML

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
import cv2

## Env setup

## Object detection imports
Here are the imports from the object detection module.

In [47]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [48]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/home/priya/Documents/AI_Apps/march_madness/faster_rcnn_inception_v2_coco_2018_01_28' + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'data/' + 'mscoco_label_map.pbtxt'

NUM_CLASSES = 80

## Load a (frozen) Tensorflow model into memory.

In [49]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [50]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [51]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [52]:
def count_nonblack_np(img):
    """Return the number of pixels in img that are not black.
    img must be a Numpy array with colour values along the last axis.

    """
    return img.any(axis=-1).sum()

In [53]:
def detect_team(image, show = False):
    # define the list of boundaries
    boundaries = [
    ([0,0,200], [255,255,255]), #white
    ([0, 0, 0], [255, 255, 60]) #black
    ]
    ratioList = []
    for (lower, upper) in boundaries:
        # create NumPy arrays from the boundaries
        lower = np.array(lower, dtype = "uint8")
        upper = np.array(upper, dtype = "uint8")

        # find the colors within the specified boundaries and apply
        # the mask
        mask = cv2.inRange(image, lower, upper)
        output = cv2.bitwise_and(image, image, mask = mask)
        tot_pix = count_nonblack_np(image)
        color_pix = count_nonblack_np(output)
        ratio = color_pix/tot_pix
        ratioList.append(ratio)
        
        if show == True:
            image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)
            cv2.imshow("images", np.hstack([image, output]))
            if cv2.waitKey(0) & 0xFF == ord('q'):
              cv2.destroyAllWindows() 
        
    ## Find the team
    max_ratio = max(ratioList)
    if max_ratio < 0.20:
        print("Not sure", ratio)
        return 'not_sure'
    else:
        ## Identify black pixels first and return them first
        if ratioList[1] >= 0.30:
#             print("Black", ratioList[0])
            return 'black'
        elif ratioList[0] >=0.20:
#             print("White", ratioList[1])
            return 'white'
        

In [54]:

## To View Color Mask
filename = '/home/priya/Documents/AI_Apps/march_madness/image1.png'
image = cv2.imread(filename)
resize = cv2.resize(image, (640,360))
hsv = cv2.cvtColor(resize, cv2.COLOR_BGR2HSV)
detect_team(hsv, show=True)

'white'

# Detection

In [18]:
filename = '/home/priya/Documents/AI_Apps/march_madness/video1_small.mp4'
out_filename = 'video1_small_out_bball' + ".avi"

cap = cv2.VideoCapture(filename)

vwriter = cv2.VideoWriter(out_filename,cv2.VideoWriter_fourcc(*'MJPG'),10, (1024, 600)) 


# Running the tensorflow session
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        
        
        counter = 0
        fac = 3
   
 
        while True:
          ret, image_np = cap.read()
          if not ret:
            break
    
          orig_image = image_np
          origW, origH, _ = orig_image.shape
            
          new_width = 1024
          new_height = 600
          image_np = cv2.resize(image_np, (new_width,new_height), interpolation=cv2.INTER_AREA)
          counter += 1
            
          h = image_np.shape[0]
          w = image_np.shape[1]

          if counter % fac == 0:          
        
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                  'num_detections', 'detection_boxes', 'detection_scores',
                  'detection_classes', 'detection_masks'
              ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image_np.shape[0], image_np.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

          # Run inference
            output_dict = sess.run(tensor_dict,
                                 feed_dict={image_tensor: np.expand_dims(image_np, 0)})

          # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
              'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
                
            ## Turned off display to avoid information overload    

#             vis_util.visualize_boxes_and_labels_on_image_array(
#               image_np,
#               output_dict['detection_boxes'],
#               output_dict['detection_classes'],
#               output_dict['detection_scores'],
#               category_index,
#               instance_masks=output_dict.get('detection_masks'),
#               use_normalized_coordinates=True,
#               min_score_thresh=0.60,
#               line_thickness=2)
            
            
            frame_number = counter
            loc = {}
            for n in range(len(output_dict['detection_scores'])):
                if output_dict['detection_scores'][n] > 0.70:
                # Calculate position
                    ymin = int(output_dict['detection_boxes'][n][0] * h)
                    xmin = int(output_dict['detection_boxes'][n][1] * w)
                    ymax = int(output_dict['detection_boxes'][n][2] * h)
                    xmax = int(output_dict['detection_boxes'][n][3] * w)

                # Find label corresponding to that class
                    for cat in categories:
                        if cat['id'] == output_dict['detection_classes'][n]:
                            label = cat['name']

                    ## extract every person
                    if label == 'person':
                        
                        ## if bbox is not too big:
                        area_box = (ymax - ymin) * (xmax - xmin)
                        area_img = h * w
                        boxPer = area_box/area_img
                        if boxPer <= 0.15:  # remove random big size detections of crowds
                            
                            ## Add them to image
                            cv2.rectangle(image_np, (xmin, ymin),(xmax,ymax),(0,255,0),3)
                        
                            #crop them
                            crop_img = image_np[ymin:ymax, xmin:xmax]
                            color = detect_team(crop_img)
                            if color != 'not_sure':
                                coords = (xmin, ymin)
                                if color == 'white':
                                     loc[coords] = 'Duke'
                                else:
                                    loc[coords] = 'UCF'
                        
            ## print color next to the person
            for key in loc.keys():
                text_pos = str(loc[key])
                cv2.putText(image_np, text_pos, (key[0], key[1]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.50, (0, 0, 255), 2) # Text in black

            vwriter.write(image_np)
#           cv2.imshow('image', image_np)
#           key = cv2.waitKey(20) & 0xFF

#                 # if the `q` key is pressed, break from the lop
#           if key == ord("q"):
#             break
            
        cv2.destroyAllWindows()
        cap.release()